In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier




In [ ]:

# 1. Carga del dataset
# Asegúrate de que el archivo 'creditcard.csv' esté en la misma carpeta
df = pd.read_csv('creditcard.csv')

# 2. Validación de Integridad
print("--- VALIDACIÓN DE DATOS ---")
print(f"1. Total de registros: {df.shape[0]}")
print(f"2. Total de columnas: {df.shape[1]}")
print(f"3. ¿Hay valores nulos?: {df.isnull().sum().sum()}")
print(f"4. ¿Hay registros duplicados?: {df.duplicated().sum()}")

# 3. Tipos de datos
# Es vital saber si el dinero ('Amount') o el tiempo ('Time') son números reales
print("\n--- TIPOS DE DATOS ---")
print(df[['Time', 'Amount', 'Class']].dtypes)

In [ ]:
# Cálculo del desbalance 
conteo = df['Class'].value_counts(normalize=True) * 100
print(f"\n--- DISTRIBUCIÓN DEL FRAUDE ---")
print(f"Transacciones Normales (0): {conteo[0]:.2f}%")
print(f"Transacciones Fraude (1): {conteo[1]:.2f}%")

In [ ]:
dperdido=df[df['Class'] == 1]['Amount'].sum()
dperdido

In [ ]:

# 1. Eliminamos duplicados como sugeriste
df.drop_duplicates(inplace=True)

# 2. Inicializamos el escalador
rob_scaler = RobustScaler()

# 3. Escalamos 'Amount' y 'Time' (las únicas que no lo están)
df['scaled_amount'] = rob_scaler.fit_transform(df['Amount'].values.reshape(-1,1))
df['scaled_time'] = rob_scaler.fit_transform(df['Time'].values.reshape(-1,1))

# 4. Quitamos las columnas originales para no confundir al modelo
df.drop(['Time','Amount'], axis=1, inplace=True)

# 5. Reorganizamos para tener las escaladas al principio (por orden)
scaled_amount = df['scaled_amount']
scaled_time = df['scaled_time']
df.drop(['scaled_amount', 'scaled_time'], axis=1, inplace=True)
df.insert(0, 'scaled_amount', scaled_amount)
df.insert(1, 'scaled_time', scaled_time)

print("Datos escalados y listos. Primeras 5 filas:")
print(df.head())

In [ ]:
# 1. Barajamos los datos antes de recortar
df = df.sample(frac=1)

# 2. Separamos fraude y normales
fraude_df = df.loc[df['Class'] == 1]
normal_df = df.loc[df['Class'] == 0][:473] # Tomamos solo 473 normales

# 3. Concatenamos para crear el dataset equilibrado
normal_dist_df = pd.concat([fraude_df, normal_df])

# 4. Volvemos a barajar para que el modelo no aprenda el orden
new_df = normal_dist_df.sample(frac=1, random_state=42)

print(f"Nuevo dataset equilibrado: {new_df.shape}")
print(new_df['Class'].value_counts())

In [ ]:

# Configuramos el lienzo
f, (ax1, ax2) = plt.subplots(2, 1, figsize=(24, 20))

# 1. Matriz de Correlación - Dataset Original (Desequilibrado)
corr = df.corr()
sns.heatmap(corr, cmap='coolwarm', annot_kws={'size':20}, ax=ax1)
ax1.set_title("Matriz de Correlación (Desequilibrada) \n No la uses para sacar conclusiones", fontsize=18)

# 2. Matriz de Correlación - Nuevo Dataset (Equilibrado)
sub_sample_corr = new_df.corr()
sns.heatmap(sub_sample_corr, cmap='coolwarm_r', annot_kws={'size':20}, ax=ax2)
ax2.set_title('Matriz de Correlación (Equilibrada) \n ¡Aquí está la información real!', fontsize=18)

plt.show()

In [ ]:
#Vamos a mirar las correlaciones para ver cual tiene mas peso en el modelo
# # Ordenados de menor a mayor
print("Variables con mayor impacto negativo:")
print(new_df.corr()['Class'].sort_values(ascending=True).head(5))

print("\nVariables con mayor impacto positivo:")
print(new_df.corr()['Class'].sort_values(ascending=False).head(5))

In [ ]:
# Analizamos V14 (la que tiene correlación negativa más fuerte)
v14_fraud = new_df['V14'].loc[new_df['Class'] == 1].values
q25, q75 = np.percentile(v14_fraud, 25), np.percentile(v14_fraud, 75)
v14_iqr = q75 - q25

# Calculamos los límites (Corte de Tukey)
v14_cut_off = v14_iqr * 1.5
v14_lower, v14_upper = q25 - v14_cut_off, q75 + v14_cut_off

print(f"Límite inferior para V14: {v14_lower}")
print(f"Límite superior para V14: {v14_upper}")

# Eliminamos los outliers del dataset
outliers = [x for x in v14_fraud if x < v14_lower or x > v14_upper]
print(f'Outliers identificados en V14: {len(outliers)}')

new_df = new_df.drop(new_df[(new_df['V14'] > v14_upper) | (new_df['V14'] < v14_lower)].index)
print('---' * 10)
print(f'Registros restantes tras limpiar V14: {len(new_df)}')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

# 1. Separamos las variables (X) de la etiqueta (y)
X = new_df.drop('Class', axis=1)
y = new_df['Class']

# 2. Dividimos el dataset (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Entrenamos el modelo
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

# 4. Hacemos las predicciones
y_pred = log_reg.predict(X_test)

print("¡Modelo entrenado y predicciones generadas!")

In [ ]:
# Visualizamos la Matriz de Confusión
conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicción (IA)')
plt.ylabel('Realidad (Banco)')
plt.title('Matriz de Confusión: ¿Qué tan buena es nuestra IA?')
plt.show()

# Resumen de métricas
print(classification_report(y_test, y_pred))

In [ ]:
#%pip install imbalanced-learn


In [ ]:
# 1. Usamos el dataset ORIGINAL 
X = df.drop('Class', axis=1)
y = df['Class']

# 2. División Crítica: Mantenemos el test con datos reales y desbalanceados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Aplicamos SMOTE solo al entrenamiento
print(f"Antes de SMOTE: {sum(y_train==1)} fraudes")
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
print(f"Después de SMOTE: {sum(y_train_res==1)} fraudes (¡Equilibrado!)")

# 4. Entrenamos la misma Regresión Logística
log_reg_smote = LogisticRegression(max_iter=1000)
log_reg_smote.fit(X_train_res, y_train_res)

# 5. Predicción sobre los datos reales de TEST
y_pred_smote = log_reg_smote.predict(X_test)



In [ ]:

# 1. Generamos la matriz de confusión para las predicciones con SMOTE
cm_smote = confusion_matrix(y_test, y_pred_smote)

# 2. Visualización
plt.figure(figsize=(7,5))
sns.heatmap(cm_smote, annot=True, fmt='d', cmap='Greens', cbar=True)
plt.xlabel('Predicción (IA con SMOTE)')
plt.ylabel('Realidad (Banco)')
plt.title('Matriz de Confusión: Modelo con Entrenamiento SMOTE')
plt.show()

# 3. Informe detallado
print("--- INFORME DE RENDIMIENTO (SMOTE) ---")
print(classification_report(y_test, y_pred_smote))

In [ ]:

# 1. Inicializar el modelo (con 100 árboles para empezar)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

# 2. Entrenar con los datos de SMOTE (los mismos que usamos antes)
print("Entrenando el Bosque Aleatorio... Esto puede tardar un poco más.")
rf_model.fit(X_train_res, y_train_res)

# 3. Predicciones
y_pred_rf = rf_model.predict(X_test)

# 4. Evaluación
print("\n--- INFORME DE RENDIMIENTO: RANDOM FOREST + SMOTE ---")
print(classification_report(y_test, y_pred_rf))

# 5. Matriz de Confusión
cm_rf = confusion_matrix(y_test, y_pred_rf)
plt.figure(figsize=(7,5))
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Purples', cbar=True)
plt.title('Matriz de Confusión: Random Forest + SMOTE')
plt.show()